In [1]:
import requests
from binascii import unhexlify, hexlify

ciphertext = "b63bd404b61da93925d3465de12c9fe78fa14dbd3ed4601d10781876d936d845"
cipher = unhexlify(ciphertext)

# Sending request to web server
payload = { 'ciphertext' : ciphertext }
response = requests.post('http://35.173.236.202:8000/crypto4/', data=payload)

known = []
skip  = 1

print(response.text)

for i in range(15, -1, -1):
    
    for j in range(0, 256): 
        x = b""
        x += hexlify((j ^ cipher[i] ^ (16 - i)).to_bytes(1, "little"))
        for m in range(0, 15 - i):
            x += hexlify((known[len(known)-m-1] ^ cipher[i + m + 1] ^ (16 - i)).to_bytes(1, "little"))

        # Build blocks
        c = ciphertext[:i * 2] + str(x)[2: (16 - i) * 2 + 2] + ciphertext[i * 2 + (16 - i) * 2:]
        #print(i, x, j, c, known)
        
        payload = { 'ciphertext' : c }
        response = requests.post('http://35.173.236.202:8000/crypto4/', data=payload)
        
        if response.status_code == 200:
            if ciphertext == c and skip == 1:
                skip = 0  
            else:
                known.append(j)
                #print(chr(j), i, x, j, c, known)
                print(chr(j), j, c)
                break    
        
        


OK
 5 b63bd404b61da93925d3465de12c9fe38fa14dbd3ed4601d10781876d936d845
 5 b63bd404b61da93925d3465de12c98e08fa14dbd3ed4601d10781876d936d845
 5 b63bd404b61da93925d3465de12a99e18fa14dbd3ed4601d10781876d936d845
 5 b63bd404b61da93925d3465de02d9ee68fa14dbd3ed4601d10781876d936d845


KeyboardInterrupt: 